In [1]:
import csv
import cv2
import numpy as np

In [2]:
"""Extract data from the .csv file"""
lines = []
with open('./data/driving_log.csv') as file:
    reader = csv.reader(file)
    for line in reader:
        lines.append(line)
# images and mearsurments will be our input_X and input_y
# Augment training set by flip the image horizonaly and flip the sign of mearsurment.
augmented_images = []
augmented_mearsurements = []
for i in range(len(lines)):
    if i == 0:
        continue
    line = lines[i]
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = './data/IMG/' + filename
    image = cv2.imread(current_path)
    augmented_images.append(image)
    augmented_images.append(cv2.flip(image, 1))
    mearsurement = float(line[3])
    augmented_mearsurements.append(mearsurement)
    augmented_mearsurements.append(mearsurement * -1.0)

In [4]:
"""Set X_train and y_train"""
X_train = np.array(augmented_images)
y_train = np.array(augmented_mearsurements)
print(X_train.shape, y_train.shape)

(16072, 160, 320, 3) (16072,)


In [7]:
"""Build a simple regression model"""
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Convolution2D, MaxPooling2D, Cropping2D

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
# crop 70 rows of pixels on the top and 25 rows of pixels on the bottom.
model.add(Cropping2D(cropping=((70, 25), (0, 0))))
model.add(Convolution2D(6, 5, 5, activation='relu'))
model.add(MaxPooling2D())
model.add(Convolution2D(16, 5, 5, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
# output regression layer
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=2)

model.save("./models/model_aug_2e.h5")

/home/nqiao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`
  if __name__ == '__main__':
/home/nqiao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (5, 5), activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
/home/nqiao/anaconda3/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 12857 samples, validate on 3215 samples
Epoch 1/2
12857/12857 [==============================] - 7s 580us/step - loss: 0.0229 - val_loss: 0.0113
Epoch 2/2
12857/12857 [==============================] - 7s 570us/step - loss: 0.0094 - val_loss: 0.0114
